# Iniciamos_CnxTestOADCloud

Realiza un testing rápido, probamos los componentes básicos de
la conexión para validar la misma.
La constante *ORA_INST_CLIENT_TNSC* debería ser sustituida por el valor de la entrada TNS de nuestra conexión de ejemplo.
En el caso de conectarnos a Oracle Cloud, las credenciales que contiene el Wallet de la BD, incorpora un fichero tnsnames.ora el las entradas a 3 niveles, low, medium y high.


In [ ]:
import sys
import cx_Oracle
import getpass
from IPython.display import HTML, display
import tabulate

# TODO Agregar soporte para modulo logging

# -- Definicion del PATH donde hemos instalado Oracle Instant Client
# -- Inicializaremmos de forma explicita
ORA_INST_CLIENT_PATH = r"C:\instantclient_19_8"
# -- Importante! Se ha de sustituir <TNS_ENTRADA>
# -- por la correspondiente a la conexion de la BD
ORA_INST_CLIENT_TNSC = r"<TNS_ENTRADA>"

# -- Solicitamos usuario y password, no recomiendo hardcode de los
# -- mismos en el codigo.
oc_user = input('AD user')
oc_pass = getpass.getpass('AD password')

# -- Ping DB Connection
# -- c = cx_Oracle connect object
# -- return True Si ping OK o False si no
# TODO (Sebas) Implmentar retries a futuro
def isConnOpen(c):
    try:
        return c.ping() is None
    except:
        return False

# -- Declaracion de conexion y cursor.
dbx = None 
dbc = None

# -- Explicitamente podemos inicializar IC pasando el 
# -- directorio de instalacion en el parametro config_dir
# -- En caso de tener activas las variables de entorno PATH y
# -- TNS_ADMIN no seria requerido.
try:
    cx_Oracle.init_oracle_client(config_dir=ORA_INST_CLIENT_PATH)
except cx_Oracle.ProgrammingError:
    print("Instant Client ya inicializado")
else:
    print("Instant Cliente inicializado correctamente")

try:
    # -- Nos conectamos a la base de datos
    dbx = cx_Oracle.connect(oc_user, oc_pass, ORA_INST_CLIENT_TNSC)

    # -- Autocommit a True
    dbx.autocommit = True

    # -- Abrir un cursor
    dbc = dbx.cursor()

    # -- Ejemplo de seteo de variable via cx_Oracle
    dbx.stmtcachesize = 40

    # -- Obtenemos algunos parametros y verificamos conectividad
    tblvers = [['Variable', 'Valor', 'Origen'],
        ["Instant Client Version",str(cx_Oracle.clientversion()), 'module'],
        ["cx_Oracle Version",cx_Oracle.version, 'module'],
        ["DBX Version ADB",dbx.version, 'connection'],
        ["DBX DSN Conexion",dbx.dsn, 'connection'],
        ["DBX Encoding", dbx.encoding, 'connection'],
        ["DBX stmtcachesize", dbx.stmtcachesize, 'connection'],
        ["DBX TNS entrada", dbx.tnsentry, 'connection'],
        ["DBX callTimeout", dbx.callTimeout, 'connection'],
        ["DBC arraysize", dbc.arraysize, 'cursor'],
        ["DBC bindarraysize", dbc.bindarraysize, 'cursor'],]

    display(HTML(tabulate.tabulate(tblvers, tablefmt='html')))

    # -- Enviamos para parseo una select simple sin ejecucion
    dbc.parse('SELECT TABLE_NAME FROM USER_TABLES')

# -- Captura de error de base de datos, a partir de aqui 
# -- extendemos la plantilla segun nuestras necesidades    
except cx_Oracle.DatabaseError as e:
    print('[ERROR] Ha ocurrido una excepcion del tipo: {0}'.format(e))
except:
    print('[ERROR] Error/Excepcion No capturada... ' + str(sys.exc_info()[0]) + + str(sys.exc_info()[1]))
else:
    print('[INFO] No se han capturado excepciones durante las conexiones')
finally:
    if(isConnOpen(dbx)):
        print('[INFO] Cerrar cursores y conexion a Oracle Cloud ADB')
        dbc.close()
        dbx.close()
    else:
        print('[WARN] No se detecta una conexion activa')
